셀 1 — 설정/유틸(원자, 전이·스펙트럼 도우미)

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, math, time
from dataclasses import dataclass
from typing import List, Tuple
from arc import Caesium, Rubidium

# ==== 사용자 설정 ====
ATOM_SPECIES = "Cs"                 # "Cs" 또는 "Rb"
THZ_BAND = (0.6e12, 1.0e12)         # [Hz] 스캔 대역
N_RANGE = (10, 40)                  # 탐색 n 범위
L_PAIRS = [("P","D"), ("D","P")]    # 후보 계열
J_OPTIONS = {"P":[1.5], "D":[2.5]}  # 빠른 스캔용 j
VISIBLE_RANGE = (350.0, 750.0)      # nm, 보기용
COUNT_WINDOW = (500.0, 600.0)       # nm, 카메라 민감대역
BIN_WIDTH_NM = 1.0                  # nm, 히스토그램 bin 폭
N_FLOOR = 5                         # 하향 탐색 최저 n'
MAX_STEPS = 12                      # 분기비 누적 캐스케이드 최대 단계
MIN_WEIGHT = 1e-6                   # 가지치기 임계
TOPK = 20                           # 상위 표시 개수
POL_Q = 0                           # THz 편광: -1(σ-), 0(π), +1(σ+)
M_J = 0.5                           # 상위 상태의 m_j (getRabiFrequency2용)

HBAR = 1.054_571_817e-34

def make_atom(name="Cs"):
    return Caesium() if name.lower() in ("cs","caesium","cesium") else Rubidium()

def l_to_letter(l):
    return "SPDFGH"[l] if l < 6 else f"L{l}"

def state_str(s):
    n,l,j = s
    return f"{n}{l_to_letter(l)}_{int(2*j)}/2"

def l_letter_to_l(L): return {"S":0,"P":1,"D":2,"F":3,"G":4,"H":5}[L]

def trans_freq(atom, s1, s2)->float:
    return abs(atom.getTransitionFrequency(*s1, *s2))  # [Hz]

def trans_lambda_nm(atom, s1, s2)->float:
    lam_m = abs(atom.getTransitionWavelength(*s1, *s2))
    return lam_m*1e9 if lam_m>0 else 0.0

def omega_1Vpm(atom, s1, s2, mj1=M_J, q=POL_Q)->float:
    # 라비 주파수 [rad/s] @ E=1 V/m (ARC helper)
    try:
        return abs(atom.getRabiFrequency2(*s1, mj1, *s2, q, 1.0))
    except Exception:
        return 0.0

def d_from_omega(omega_rad_s: float, E=1.0)->float:
    # |d| [C·m], 표기용
    return HBAR*omega_rad_s/E


셀 2 — 분기비 누적 스펙트럼 & 500–600 nm 수율

In [ ]:
def allowed_transitions_with_A(atom, s, n_floor=N_FLOOR):
    """상태 s=(n,l,j)의 가능한 E1 하향 전이와 (A_ij, λ_nm)를 나열."""
    n,l,j = s
    outs = []
    for lp in (l-1, l+1):
        if lp < 0: continue
        for jp in (lp-0.5, lp+0.5):
            if jp <= 0: continue
            for np_ in range(max(n_floor,5), n):
                try:
                    A = atom.getTransitionRate(n,l,j, np_,lp,jp)  # [s^-1]
                except Exception:
                    A = 0.0
                if A > 0.0:
                    lam_nm = trans_lambda_nm(atom, s, (np_,lp,jp))
                    if lam_nm>0: outs.append(((np_,lp,jp), A, lam_nm))
    return outs  # [(next_state, Aij, λ_nm), ...]

def branching_spectrum(atom, start,
                       max_steps=MAX_STEPS, min_weight=MIN_WEIGHT, n_floor=N_FLOOR):
    """
    초기 여기 1회당 기대 스펙트럼(무작위 없음).
    반환: DataFrame with columns ["lambda_nm","weight"]
    """
    lines = []
    stack = [(start, 1.0, 0)]
    while stack:
        s, w, depth = stack.pop()
        if w < min_weight or depth >= max_steps: 
            continue
        # 낮은 준위면 종료(n<=7 & l<=1 가드)
        if s[0] <= 7 and s[1] <= 1:
            continue
        trs = allowed_transitions_with_A(atom, s, n_floor=n_floor)
        if not trs: 
            continue
        A_sum = sum(A for _,A,_ in trs)
        if A_sum <= 0:
            continue
        for s2, Aij, lam_nm in trs:
            p = Aij/A_sum
            w_child = w*p
            lines.append((lam_nm, w_child))  # 해당 전이에서 광자 1개 기대
            stack.append((s2, w_child, depth+1))
    if not lines:
        return pd.DataFrame(columns=["lambda_nm","weight"])
    df = pd.DataFrame(lines, columns=["lambda_nm","weight"])
    # 동일 파장(반올림 bin) 병합
    df = df.groupby("lambda_nm", as_index=False)["weight"].sum()
    return df

def visible_yield_500_600(df_lines: pd.DataFrame, window_nm=COUNT_WINDOW)->float:
    a,b = window_nm
    return df_lines.query(f"{a} <= lambda_nm <= {b}")["weight"].sum()

def plot_spectrum(df_lines, start_state, title_suffix=""):
    a,b = VISIBLE_RANGE
    bins = np.arange(a, b+BIN_WIDTH_NM, BIN_WIDTH_NM)
    hist, edges = np.histogram(df_lines["lambda_nm"], bins=bins, weights=df_lines["weight"])
    centers = 0.5*(edges[:-1]+edges[1:])
    fig, ax = plt.subplots(figsize=(8,4))
    ax.bar(centers, hist, width=BIN_WIDTH_NM, align="center")
    ax.set_xlabel("Wavelength (nm)")
    ax.set_ylabel("Expected photons per excitation")
    ax.set_title(f"Branching-ratio spectrum from {state_str(start_state)}{title_suffix}")
    ax.set_xlim(a, b)
    ax.grid(True, alpha=0.2)
    plt.show()


셀 3 — 0.6–1.0 THz 후보 스캔/랭킹(분기비 수율 결합)

In [ ]:
@dataclass
class Candidate:
    upper: Tuple[int,int,float]
    lower: Tuple[int,int,float]
    f_THz: float
    Omega_1Vpm: float
    d_Cm_from_Omega: float
    yield_best_500_600: float

def scan_candidates_with_branching(atom_name=ATOM_SPECIES)->List[Candidate]:
    atom = make_atom(atom_name)
    out: List[Candidate] = []
    fmin, fmax = THZ_BAND
    nmin, nmax = N_RANGE
    t0 = time.time()
    for L1, L2 in L_PAIRS:
        l1, l2 = l_letter_to_l(L1), l_letter_to_l(L2)
        for j1 in J_OPTIONS[L1]:
            for j2 in J_OPTIONS[L2]:
                for n1 in range(nmin, nmax+1):
                    for dn in (0, -1, +1, -2, +2):
                        n2 = n1 + dn
                        if n2 < nmin or n2 > nmax or n2 == n1:
                            continue
                        s1, s2 = (n1,l1,j1), (n2,l2,j2)
                        f = trans_freq(atom, s1, s2)
                        if not (fmin <= f <= fmax): 
                            continue
                        # THz 결합 강도
                        omega = omega_1Vpm(atom, s1, s2)      # [rad/s] @ 1 V/m
                        d_cm = d_from_omega(omega, 1.0)       # [C·m] 표기용
                        # 분기비 누적 스펙트럼 → 500–600 nm 수율 (상/하 중 큰 값)
                        df1 = branching_spectrum(atom, s1)
                        df2 = branching_spectrum(atom, s2)
                        y1 = visible_yield_500_600(df1)
                        y2 = visible_yield_500_600(df2)
                        out.append(Candidate(
                            upper=s1, lower=s2, f_THz=f,
                            Omega_1Vpm=omega, d_Cm_from_Omega=d_cm,
                            yield_best_500_600=max(y1,y2)
                        ))
    print(f"scan done: {len(out)} candidates in {time.time()-t0:.1f}s")
    return out

def score(c: Candidate)->float:
    # FoM = Ω(1V/m) × (분기비 누적 기반 500–600 nm 수율)
    return abs(c.Omega_1Vpm) * c.yield_best_500_600

def fmt_state(s):
    n,l,j = s
    return f"{n}{l_to_letter(l)}_{int(2*j)}/2"

def to_dataframe(cands: List[Candidate], topk=TOPK)->pd.DataFrame:
    rows = []
    for c in sorted(cands, key=score, reverse=True)[:topk]:
        rows.append({
            "upper": fmt_state(c.upper),
            "lower": fmt_state(c.lower),
            "f_THz": c.f_THz/1e12,
            "Ω(1 V/m) [Hz]": c.Omega_1Vpm/(2*math.pi),
            "|d| [C·m] (from Ω)": c.d_Cm_from_Omega,
            "yield_500-600nm": c.yield_best_500_600,
            "FoM = Ω × yield": score(c)
        })
    return pd.DataFrame(rows)

# 실행
cands = scan_candidates_with_branching(ATOM_SPECIES)
df = to_dataframe(cands, topk=TOPK)
df


셀 4 — 상위 후보 스펙트럼 플롯(분기비 누적)

In [ ]:
atom = make_atom(ATOM_SPECIES)

def plot_top_k_spectra(df_ranked: pd.DataFrame, k=3):
    for i in range(min(k, len(df_ranked))):
        row = df_ranked.iloc[i]
        # 문자열 상태 다시 파싱 대신, 원본 Candidate를 찾는 편이 안전
        # 여기서는 간단히 다시 재계산
        # (상태 문자열을 파싱하려면 별도 파서 필요하므로, 아래는 재탐색)
        print(f"#{i+1}  {row['upper']} ↔ {row['lower']}  | f={row['f_THz']:.3f} THz")
    print("\n개별 플롯은 원본 Candidate 객체에서 시작 상태를 넘겨 그리세요.")

# 예시: 상위 1개 후보의 '상'과 '하' 상태 스펙트럼 각각 보기
if len(cands) > 0:
    best = sorted(cands, key=score, reverse=True)[0]
    df_up = branching_spectrum(atom, best.upper)
    df_lo = branching_spectrum(atom, best.lower)
    print(f"Best candidate: {fmt_state(best.upper)} ↔ {fmt_state(best.lower)}  | f={best.f_THz/1e12:.3f} THz")
    print(f"yield(500–600 nm): upper={visible_yield_500_600(df_up):.4f}, lower={visible_yield_500_600(df_lo):.4f}")
    plot_spectrum(df_up, best.upper, title_suffix=" (upper)")
    plot_spectrum(df_lo, best.lower, title_suffix=" (lower)")


셀 1→2→3 실행하면 **랭킹 표(df)**가 뜹니다.

셀 4로 상위 후보 스펙트럼을 바로 확인하세요.

COUNT_WINDOW를 바꿔(예: 520±10 nm) 필터/카메라 조건에 맞게 수율 기준을 즉시 재랭킹할 수 있습니다.

POL_Q, M_J를 바꿔 편광·m_j 조합에 따른 Ω(결합 강도) 최적화를 볼 수 있습니다.

셀 — CSV 저장 유틸 (랭킹/풀테이블/스펙트럼까지)

In [ ]:
# CSV export utilities for the Rydberg THz scan
from pathlib import Path
from datetime import datetime
import json, math, pandas as pd
  
def export_results_csv(export_dir="exports", base_name=None, save_topk=TOPK, save_spectra_k=3):
    """
    저장 항목:
      1) 랭킹 TopK:   <base>_ranking_topK.csv
      2) 전체 후보:   <base>_ranking_full.csv
      3) 스펙트럼(상위 k개 후보의 상/하 상태 각각):
                      <base>_top{i}_{STATE}_spectrum.csv
      4) 메타데이터:  <base>_meta.json
    반환: 파일 경로 dict
    """
    # 사전 체크
    if "df" not in globals() or "cands" not in globals():
        raise RuntimeError("먼저 후보 스캔을 수행해 df/cands를 만들어 주세요.")

    atom = make_atom(ATOM_SPECIES)

    # 베이스 이름
    if base_name is None:
        tstamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_name = f"{ATOM_SPECIES}_scan_{THZ_BAND[0]/1e12:.1f}-{THZ_BAND[1]/1e12:.1f}THz_{tstamp}"

    outdir = Path(export_dir)
    outdir.mkdir(parents=True, exist_ok=True)

    # 1) 랭킹 TopK (현재 df가 TopK 테이블이라면 그대로 저장)
    f_rank = outdir / f"{base_name}_ranking_top{save_topk}.csv"
    df.to_csv(f_rank, index=False)

    # 2) 전체 후보 풀 테이블 만들기
    def cand_to_row(c):
        return {
            "upper": fmt_state(c.upper),
            "lower": fmt_state(c.lower),
            "f_THz": c.f_THz/1e12,
            "Omega_1Vpm_Hz": c.Omega_1Vpm/(2*math.pi),
            "|d|_from_Omega_Cm": c.d_Cm_from_Omega,
            "yield_500-600nm": c.yield_best_500_600,
            "FoM=Omega*yield": score(c)
        }
    df_full = (pd.DataFrame([cand_to_row(c) for c in cands])
                 .sort_values("FoM=Omega*yield", ascending=False)
                 .reset_index(drop=True))
    f_full = outdir / f"{base_name}_ranking_full.csv"
    df_full.to_csv(f_full, index=False)

    # 3) 스펙트럼 파일(상위 k개 후보의 upper/lower 각각)
    files_spectra = []
    top_list = sorted(cands, key=score, reverse=True)[:save_spectra_k]
    for i, c in enumerate(top_list, 1):
        df_up = branching_spectrum(atom, c.upper)
        df_lo = branching_spectrum(atom, c.lower)
        f_up = outdir / f"{base_name}_top{i}_{fmt_state(c.upper)}_spectrum.csv"
        f_lo = outdir / f"{base_name}_top{i}_{fmt_state(c.lower)}_spectrum.csv"
        df_up.to_csv(f_up, index=False)
        df_lo.to_csv(f_lo, index=False)
        files_spectra += [str(f_up), str(f_lo)]

    # 4) 메타데이터 저장
    meta = {
        "atom": ATOM_SPECIES,
        "thz_band_Hz": THZ_BAND,
        "n_range": N_RANGE,
        "l_pairs": L_PAIRS,
        "j_options": J_OPTIONS,
        "count_window_nm": COUNT_WINDOW,
        "visible_range_nm": VISIBLE_RANGE if "VISIBLE_RANGE" in globals() else None,
        "bin_width_nm": BIN_WIDTH_NM if "BIN_WIDTH_NM" in globals() else None,
        "polarization_q": POL_Q if "POL_Q" in globals() else None,
        "mj1_used": M_J if "M_J" in globals() else None,
        "topk": int(save_topk),
        "spectra_k": int(save_spectra_k),
        "timestamp": datetime.now().isoformat(timespec="seconds")
    }
    f_meta = outdir / f"{base_name}_meta.json"
    with open(f_meta, "w", encoding="utf-8") as fp:
        json.dump(meta, fp, ensure_ascii=False, indent=2)

    return {
        "ranking_top": str(f_rank),
        "ranking_full": str(f_full),
        "spectra": files_spectra,
        "meta": str(f_meta)
    }

# 사용 예:
# paths = export_results_csv(export_dir="exports", save_topk=TOPK, save_spectra_k=3)
# paths
